# Project

## Imports

In [1]:
pip install config

In [2]:
pip install 'tweepy==4.4.0'

In [7]:
import numpy as np
import pandas as pd
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import tensorflow as tf
from sklearn import preprocessing, model_selection
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

import tweepy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
FILE_PATH = '/content/drive/MyDrive/ML/GBC/DL1/Project/'

## Load Tweets

In [11]:
# import connection info
twitter_api_info = pd.read_csv(FILE_PATH + 'CONNECTION_INFO.csv')
twitter_api_info = twitter_api_info.set_index('key')

In [12]:
API_KEY = twitter_api_info.loc['API_KEY']['value']
API_KEY_SECRET = twitter_api_info.loc['API_KEY_SECRET']['value']
BEARER_TOKEN = twitter_api_info.loc['BEARER_TOKEN']['value']
ACCESS_TOKEN = twitter_api_info.loc['ACCESS_TOKEN']['value']
ACCESS_TOKEN_SECRET = twitter_api_info.loc['ACCESS_TOKEN_SECRET']['value']

In [13]:
# Functions to grab tweets

def getClient():
    client = tweepy.Client(
        bearer_token=BEARER_TOKEN,
        consumer_key=API_KEY,
        consumer_secret=API_KEY_SECRET,
        access_token=ACCESS_TOKEN,
        access_token_secret=ACCESS_TOKEN_SECRET
    )
    return client

def searchTweets(client, query, max_results):
    tweets = client.search_recent_tweets(query=query, max_results=max_results)

    tweet_data =  tweets.data
    results = []

    if not tweet_data is None and len(tweet_data) > 0:
        for tweet in tweet_data:
            obj = {}
            obj['id'] = tweet.id
            obj['text'] = tweet.text
            results.append(obj)

    return results

def returnTweets(query):
    query = '{} lang:en -is:retweet'.format(query)

    client = getClient()
    tweets = searchTweets(client, query, 50)

    objs = []

    if len(tweets) > 0:
        for tweet in tweets:
            obj = {}
            obj['text'] = tweet['text']
            objs.append(obj)

    return(objs)

In [14]:
tweets = searchTweets(getClient(), 'bitcoin', 10)
# tweets
for x in tweets:
    print(x)

# query = 'canada'
# returnTweets(query)

{'id': 1485819882667397124, 'text': 'RT @BitcoinMagazine: "The safest and most profitable moment to buy is when the price is down." – El Salvador president on #Bitcoin https://…'}
{'id': 1485819882067349508, 'text': 'Trending on #LunarCrush:\n\n"Investidores aproveitam queda do Bitcoin e investem US$ 14 milhões em fundos de criptomoedas" via @infomoney\n\nTop coin mentions\n$btc\n\nhttps://t.co/wuzaPWGHIE'}
{'id': 1485819880116998146, 'text': 'RT @freecashcom: $100.00 #Bitcoin Giveaway\n\n- Retweet\n- Follow us\n\nRolling in 24 hours 🔥'}
{'id': 1485819878812819456, 'text': 'I am in the running to win the Metaverse Lab Airdrop with the Prizepool of 20,000 $MVP ,Check it out #Giveaway #Airdrop #BNB #Solana #Bitcoin https://t.co/YClXFm7hGU'}
{'id': 1485819876065386496, 'text': '#Bitcoin es y será siempre un resguardo de valor. Siempre será mejor tener un registro mundial descentralizado que confiar en instituciones corrompibles. 👀 https://t.co/mcFabG6qDR'}
{'id': 1485819875834855424, 'tex

## Sentiment Analysis

In [16]:
# Load Stop words and add capital I
stopwords = stopwords.words('english')
new_words=['I']
for i in new_words:
    stopwords.append(i)

In [18]:
# Load the data set
# You may need pandas >1.3.0 for "on_bad_lines" to work
data = pd.read_csv(FILE_PATH + '/sentiment_analysis_dataset.csv', sep=',', encoding='latin-1', on_bad_lines='skip')

In [19]:
# Remove all the stop words inn the dataset
data['X_Clean'] = data['SentimentText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

# Define X & Y
Y = data.Sentiment
X = data.X_Clean


X = X.iloc[:10000]
Y = Y.iloc[:10000]


In [20]:
# only work with the 3000 most popular words found in our dataset
max_words = 2000

# create a new Tokenizer
tokenizer = Tokenizer(num_words=max_words)

# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(X)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index


with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)
    

def convert_text_to_index_array(text):
    #  text_to_word_sequence makes all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]



allIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in X:
    wordIndices = convert_text_to_index_array(text)
    allIndices.append(wordIndices)
    
    
# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allIndices = np.asarray(allIndices,dtype=object)  
    
# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allIndices, mode='binary')
# treat the labels as categories
train_y = tf.keras.utils.to_categorical(
    Y, num_classes=2, dtype='float32')

train_x, test_x, train_y, test_y = model_selection.train_test_split(train_x,train_y,test_size = 0.2, random_state = 0)

model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
# model.add(Dropout(0.1))
model.add(Dense(256, activation='sigmoid'))
# model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))
    
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])   
    

model.fit(train_x, train_y,
  batch_size=80,
  epochs=5,
  validation_split=0.1,
  shuffle=True)

Epoch 1/5
90/90 [==============================] - 2s 5ms/step - loss: 0.5916 - accuracy: 0.6760 - val_loss: 0.5293 - val_accuracy: 0.7350
Epoch 2/5
90/90 [==============================] - 0s 3ms/step - loss: 0.3669 - accuracy: 0.8331 - val_loss: 0.5255 - val_accuracy: 0.7563
Epoch 3/5
90/90 [==============================] - 0s 3ms/step - loss: 0.2348 - accuracy: 0.9076 - val_loss: 0.6329 - val_accuracy: 0.7412
Epoch 4/5
90/90 [==============================] - 0s 3ms/step - loss: 0.1422 - accuracy: 0.9475 - val_loss: 0.7592 - val_accuracy: 0.7450
Epoch 5/5
90/90 [==============================] - 0s 3ms/step - loss: 0.0853 - accuracy: 0.9692 - val_loss: 0.8538 - val_accuracy: 0.7475
